In [530]:
import pandas as pd
from datetime import datetime, timedelta
from random import randrange
import s3fs
from pyathena import connect
from pandasql import sqldf
import pyarrow as pa
import pyarrow.parquet as pq

In [531]:
#df_new_file = pd.read_csv('s3://stock-market-raw-data-us-east-1/stg_price_by_date/test_data.csv')
df_new_file = pd.read_csv('s3://stock-market-raw-data-us-east-1/stg_price_by_date/20250130000719.csv')

In [532]:
conn = connect(s3_staging_dir='s3://de-youtube-project-useast-dev/',region_name='us-east-1')
df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)

<ipython-input-532-8357164e96e7>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)


In [533]:
df_new_records = sqldf(
         '''select a.company company, a.date as close_date, a.close_price   
            from df_new_file a 
            left join df_athena b on a.company = b.company and 
                a.date = b.close_date
            where b.company is null
         '''
     )

In [534]:
#convert string to date python date object

df_new_records['close_date'] = df_new_records['close_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 

#OR

#df_new_records['close_date'] = pd.to_datetime(df_new_records['close_date']).dt.date 

In [535]:
#create the partition column
df_new_records['p_year'] = pd.to_datetime(df_new_records['close_date']).dt.year

In [536]:
# Convert the DataFrame to a PyArrow Table
table = pa.Table.from_pandas(df_new_records)

In [537]:
# Write to Parquet with partitioning by year
s3_path = 's3://stock-market-raw-data-us-east-1/price_by_date/'
pq.write_to_dataset(table, root_path = s3_path, partition_cols=['p_year'])

In [539]:
def process_batch_files(path):
    
    import pandas as pd
    from datetime import datetime, timedelta
    from random import randrange
    import s3fs
    from pyathena import connect
    from pandasql import sqldf
    import pyarrow as pa
    import pyarrow.parquet as pq
    
    df_new_file = pd.read_csv(path)
    
    conn = connect(s3_staging_dir='s3://de-youtube-project-useast-dev/',region_name='us-east-1')
    df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)
    
    df_new_records = sqldf(
         '''select a.company company, a.date as close_date, a.close_price   
            from df_new_file a 
            left join df_athena b on a.company = b.company and 
                a.date = b.close_date
            where b.company is null
         '''
     )
    
    df_new_records['close_date'] = df_new_records['close_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 
    df_new_records['p_year'] = pd.to_datetime(df_new_records['close_date']).dt.year
    
    table = pa.Table.from_pandas(df_new_records)
    s3_path = 's3://stock-market-raw-data-us-east-1/price_by_date/'
    pq.write_to_dataset(table, root_path = s3_path, partition_cols=['p_year'])
 

In [541]:
path = 's3://stock-market-raw-data-us-east-1/stg_price_by_date/20250131163557.csv'
process_batch_files(path)

<ipython-input-539-04455442d491>:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_athena = pd.read_sql('select * from AwsDataCatalog.stock_market.price_by_date', conn)
